<a href="https://colab.research.google.com/github/myazzeh/NLP-Course/blob/main/Transformers%20/NLP_Transformer_FineTuning_CLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
import pandas as pd

#**Download and read your Datasets**##

In [ ]:
!wget https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/fake_news_train.csv
!wget https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/fake_news_test.csv

--2024-05-21 16:41:43--  https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/fake_news_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296552 (290K) [text/plain]
Saving to: ‘fake_news_train.csv’

fake_news_train.csv 100%[===================>] 289.60K  --.-KB/s    in 0.007s  

2024-05-21 16:41:44 (40.4 MB/s) - ‘fake_news_train.csv’ saved [296552/296552]

--2024-05-21 16:41:44--  https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/fake_news_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [ ]:
data = load_dataset("csv", data_files={"train": "/content/fake_news_train.csv", "test":"/content/fake_news_test.csv"})
data

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['claim_s', 'fake_flag'],
        num_rows: 3185
    })
    test: Dataset({
        features: ['claim_s', 'fake_flag'],
        num_rows: 456
    })
})

#**Tokenize training and testing dataset**#

In [ ]:
#checkpoint = "CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment"
checkpoint= 'CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["claim_s"], truncation=True)

tokenized_datasets = data.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3185 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/456 [00:00<?, ? examples/s]

#**Convert Datasets to Tensorflow datasets**#

In [ ]:
#Dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["fake_flag"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["fake_flag"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

#**FineTune the transformer, re-train it**#

In [ ]:
from keras.losses import SparseCategoricalCrossentropy
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=10
)

tf_model.h5:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some layers from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Epoch 1/10
399/399 [==============================] - 128s 131ms/step - loss: 0.7153 - accuracy: 0.6173 - val_loss: 0.6708 - val_accuracy: 0.6711
Epoch 2/10
399/399 [==============================] - 38s 95ms/step - loss: 0.6661 - accuracy: 0.6436 - val_loss: 0.6361 - val_accuracy: 0.6711
Epoch 3/10
399/399 [==============================] - 39s 97ms/step - loss: 0.6954 - accuracy: 0.6141 - val_loss: 0.7854 - val_accuracy: 0.3289
Epoch 4/10
399/399 [==============================] - 37s 92ms/step - loss: 0.6954 - accuracy: 0.6185 - val_loss: 0.7227 - val_accuracy: 0.6711
Epoch 5/10
399/399 [==============================] - 37s 94ms/step - loss: 0.6706 - accuracy: 0.6352 - val_loss: 0.6510 - val_accuracy: 0.6711
Epoch 6/10
399/399 [==============================] - 37s 93ms/step - loss: 0.6764 - accuracy: 0.6402 - val_loss: 0.6337 - val_accuracy: 0.6711
Epoch 7/10
399/399 [==============================] - 38s 95ms/step - loss: 0.6593 - accuracy: 0.6505 - val_loss: 0.7380 - val_accurac

In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]

57/57 [==============================] - 11s 36ms/step


In [ ]:
preds

array([[  1.0412606 ,  -0.0883652 , -10.165736  ],
       [  1.0412606 ,  -0.08836518, -10.165736  ],
       [  1.0412606 ,  -0.0883652 , -10.165736  ],
       ...,
       [  1.0412606 ,  -0.0883652 , -10.165736  ],
       [  1.0412605 ,  -0.08836522, -10.165735  ],
       [  1.0412605 ,  -0.08836526, -10.165734  ]], dtype=float32)

In [ ]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(456, 3) (456,)


In [ ]:
class_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,